<a href="https://colab.research.google.com/github/sowmyarshetty/CapstoneProject-AI-Class/blob/main/AmazonHomeKitchenReviewsFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install python-dotenv
!pip install langchain_openai
!pip install langchain
!pip install langchain_community
!pip install jq
!pip install gdown
!pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 16.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import dask.dataframe as dd
import gdown
import re
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
# from bertopic import BERTopic
# from sentence_transformers import SentenceTransformer

In [3]:
# Mount Google Drive (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
amazonhkdatasetfileid = '14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o'
amazonhkdatasetfilename = 'AmazonHomeKitchenReviews.csv'

url = f"https://drive.google.com/uc?export=download&id={amazonhkdatasetfileid}"

gdown.download(url,amazonhkdatasetfilename, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?export=download&id=14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o
From (redirected): https://drive.google.com/uc?export=download&id=14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o&confirm=t&uuid=965bb08d-6789-4864-b36c-a75bb85a7b91
To: /content/AmazonHomeKitchenReviews.csv
100%|██████████| 692M/692M [00:06<00:00, 99.4MB/s]


'AmazonHomeKitchenReviews.csv'

* Read the dataset csv  into dataframes

In [5]:
df_data = pd.read_csv(amazonhkdatasetfilename)


In [8]:
print(len(df_data))

754079


In [9]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754079 entries, 0 to 754078
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         754079 non-null  int64  
 1   rating             754079 non-null  int64  
 2   title_x            753916 non-null  object 
 3   text               753888 non-null  object 
 4   images             754079 non-null  object 
 5   asin               754079 non-null  object 
 6   parent_asin        754079 non-null  object 
 7   user_id            754079 non-null  object 
 8   timestamp          754079 non-null  object 
 9   helpful_vote       754079 non-null  int64  
 10  verified_purchase  754079 non-null  bool   
 11  title_y            754079 non-null  object 
 12  description        754079 non-null  object 
 13  price              754079 non-null  float64
 14  Brand              754077 non-null  object 
 15  Material           754079 non-null  object 
 16  Co

In [ ]:
#reduce the dataset to 10000 records
# df_data = df_data.head(10000)



*   Analyse the datasets
*  Check total number of unique products and the review counts




In [10]:
df_renamed = df_data.rename(columns={'title_y' : 'product_title','title_x':'review_title','text':'review_text'})
df_renamed.groupby('product_title').size().sort_values(ascending=False).head(5)
print(df_renamed.columns)

Index(['Unnamed: 0', 'rating', 'review_title', 'review_text', 'images', 'asin',
       'parent_asin', 'user_id', 'timestamp', 'helpful_vote',
       'verified_purchase', 'product_title', 'description', 'price', 'Brand',
       'Material', 'Color', 'categories'],
      dtype='object')


* Pre-processing
* X = review_title,review_text
* y = rating

In [16]:
print(df_renamed.value_counts('price'))




price
19.99      25494
14.99      23353
29.99      21455
24.99      21141
34.97      20431
           ...  
1366.76        1
1529.90        1
1605.14        1
6.30           1
6.44           1
Name: count, Length: 3013, dtype: int64


In [22]:
qa_pairs = []
for index , row in df_renamed.iterrows():
  context = str(row['review_title']) + str(row["review_text"])

    # Generate questions about user rating only if rating is not null
  if not pd.isna(row['rating']):
        qa_pairs.append({
            'context': context,
            'question': f"What was the average rating for product{str(row['product_title'])}?",
            'answer': {'text': str(row['rating']), 'answer_start': context.find(str(row['rating'])) if str(row['rating']) in context else 0}
        })

        # Question about why they gave that rating
        qa_pairs.append({
            'context': context,
            'question': f"Why did the customer rate this product {str(row['rating'])} stars?",
             'answer': {'text': context[:150], 'answer_start': 0}
        })



In [24]:
print(qa_pairs[5:12])

[{'context': 'Pretty colors availableNice thin placemats of good size. Can be used also as table doilies.', 'question': 'Why did the customer rate this product 5 stars?', 'answer': {'text': 'Pretty colors availableNice thin placemats of good size. Can be used also as table doilies.', 'answer_start': 0}}, {'context': 'Nice materialVery pretty, wish they came bigger', 'question': 'What was the average rating for productPaperLanternStore.com 7 Inch Bloom Shaped Handmade Cotton Crochet Doilies - Beige (2 PACK)?', 'answer': {'text': '4', 'answer_start': 0}}, {'context': 'Nice materialVery pretty, wish they came bigger', 'question': 'Why did the customer rate this product 4 stars?', 'answer': {'text': 'Nice materialVery pretty, wish they came bigger', 'answer_start': 0}}, {'context': 'Love the zipper!The red is a deeper red rather than a bright red. Not as shiny as the polyester satin ones. Love the zipper enclosure since it’s a slippery material, the pillows won’t come out.', 'question': 'W

In [25]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0
